# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os
import csv

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [9]:
# Load in file
globalcities = "../../cities.csv"
globalcities_df = pd.read_csv(globalcities, encoding= "utf-8")
globalcitiesnew_df = globalcities_df.rename(columns={'Unnamed: 0': 'City Name'})
globalcitiesnew_df.head()

,City Name,Temperature,Max Temp,Latitude,Longitude,Humidity,Cloudiness,Windspeed
0,rikitea,76.75,76.75,-23.12,-134.97,82.0,100.0,4.65
1,praia da vitoria,66.20,66.20,38.73,-27.07,88.0,20.0,16.22
2,katsuura,69.69,70.00,35.13,140.30,78.0,89.0,3.00
3,mahajanga,75.33,75.33,-15.72,46.32,71.0,4.0,8.43
4,qaanaaq,27.10,27.10,77.48,-69.36,92.0,0.0,1.63


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [11]:
# Store 'Lat' and 'Lng' into  locations 
locations = globalcitiesnew_df[["Latitude", "Longitude"]].astype(float)
locations

# Convert humidity to float and store
humidity = globalcitiesnew_df["Humidity"].astype(float)
humidity
# Create a Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
idealCities_df = globalcitiesnew_df[(globalcitiesnew_df['Max Temp'] > 68) & (globalcitiesnew_df['Max Temp'] < 82) & (globalcitiesnew_df['Humidity'] < 75) & (globalcitiesnew_df['Cloudiness'] == 0) & (globalcitiesnew_df['Windspeed'] < 5)]
idealCities_df


,City Name,Temperature,Max Temp,Latitude,Longitude,Humidity,Cloudiness,Windspeed
74,pailon,69.80,69.80,-17.65,-62.75,68.0,0.0,4.70
139,agadez,79.16,79.16,19.75,10.25,14.0,0.0,1.88
140,luderitz,69.80,69.80,-26.65,15.16,26.0,0.0,3.36
168,doha,79.54,81.00,25.29,51.53,69.0,0.0,3.36
278,marzuq,79.07,79.07,14.40,46.47,43.0,0.0,1.43
284,langarud,69.76,69.76,37.20,50.15,52.0,0.0,1.77
289,namibe,77.20,77.20,-15.20,12.15,50.0,0.0,2.91
378,nioro,77.77,77.77,13.35,-15.75,41.0,0.0,4.65
487,nushki,69.19,69.19,29.55,66.02,22.0,0.0,2.46


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:
#create a variable to store the hotels list
hotel_df = []
#add a hotel name column to the existing dataframe
idealCities_df["Hotel Name"] = ""
idealCities_df

C:\Users\mcclenk\Anaconda3\envs\PythonDataViz\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,City Name,Temperature,Max Temp,Latitude,Longitude,Humidity,Cloudiness,Windspeed,Hotel Name
74,pailon,69.8,69.8,-17.65,-62.75,68,0,4.7,
139,agadez,79.16,79.16,19.75,10.25,14,0,1.88,
140,luderitz,69.8,69.8,-26.65,15.16,26,0,3.36,
168,doha,79.54,81,25.29,51.53,69,0,3.36,
278,marzuq,79.07,79.07,14.4,46.47,43,0,1.43,
284,langarud,69.76,69.76,37.2,50.15,52,0,1.77,
289,namibe,77.2,77.2,-15.2,12.15,50,0,2.91,
378,nioro,77.77,77.77,13.35,-15.75,41,0,4.65,
487,nushki,69.19,69.19,29.55,66.02,22,0,2.46,
Hotel Name,,,,,,,,,


In [26]:
#hint: use iterrows to loop through the dataset to search for hotels <5000 meters from lat and long 

params = {
    "units": "metric", 
    "location": = f"{'Latitude'},{'Longitude'}"
    "distance": 5000, 
    "type": "restaurant", 
    "key": g_key
}


# use iterrows to iterate through pandas dataframe
for index, row in idealCities_df.iterrows():
    #lat = row['Latitude']
    #lng = row['Longitude']
    #params['location'] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
   
    
    try:
        print(f"Hotel Name is {response['results'][0]['name']}.")
        
        idealCities_df.loc[index, '']
        
        
    except (KeyError, IndexError):
        print("KeyError")
        

SyntaxError: invalid syntax (<ipython-input-26-fdf17797a5f3>, line 5)

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
